Import public packages and functions

In [2]:
import os
import importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from itertools import compress


import openpyxl
from datetime import datetime
from scipy.spatial import distance
from scipy.ndimage import uniform_filter1d
import math
from scipy.signal import find_peaks, peak_prominences
import statistics as stat
import json


In [4]:
def get_repo_path_in_notebook():
    """
    Finds path of repo from Notebook.
    Start running this once to correctly find
    other modules/functions
    """
    path = os.getcwd()
    repo_name = 'ultraleap_analysis'

    while path[-len(repo_name):] != 'ultraleap_analysis':

        path = os.path.dirname(path)

    return path


In [5]:
repo_path = get_repo_path_in_notebook()
code_path = os.path.join(repo_path, 'code')
os.chdir(code_path)


Import own functions

In [6]:
import import_data.import_and_convert_data as import_dat
import import_data.find_paths as find_paths
import import_data.preprocessing_meta_info as meta_info
import sig_processing.segment_tasks as tasks
import movement_calc.calc_3d_distances as calc_3d
import movement_calc.helpfunctions as hp
import sig_processing.extract_features as feat_ext


### Saving blocks for preprocessing

Reloading own functions

In [7]:
importlib.reload(import_dat)
importlib.reload(tasks)
importlib.reload(find_paths)
importlib.reload(feat_ext)

<module 'sig_processing.extract_features' from 'c:\\Users\\madal\\Thesis\\ultraleap_analysis\\code\\sig_processing\\extract_features.py'>

Define variables of interest

In [8]:
ls_cam = ['dt', 'vr']
task = 'oc'
ls_cond = ['m1s0', 'm1s1']
ls_side = ['left', 'right']
sub = 'ul004'
folder = 'patientdata'


Save blocks for feature extraction

In [10]:
for side in ls_side:
    for cam in ls_cam:
        for cond in ls_cond:
            # get defined data
            data = import_dat.get_data(
                sub=sub, cam_pos=cam, task=task, condition=cond, side=side,
            )

            # get block dataframes
            blocks = tasks.block_extraction(
                data, sub=sub, task=task, side=side,
                cond=cond, cam=cam, to_save=True
            )
            block_path = os.path.join(
                repo_path, 'data', 'raw_blocks', f'ul{sub}', task, cond, 'blocks')

            # saving blocks as .csv files
            for block in blocks.keys():
                if blocks[block].empty:
                    print(f'{block}_ul{sub}_{cond}_{cam}_{task}_{side} is empty')
                    continue
                if not os.path.exists(block_path) and not blocks[block].empty:
                    os.makedirs(block_path)
                blocks[block].to_csv(os.path.join(
                    block_path, f'{block}_ul{sub}_{cond}_{cam}_{task}_{side}.csv'))

                if task == 'oc':
                    dist = hp.calc_distances(
                        blocks[block], ['index_tip', 'middle_tip', 'ring_tip', 'pinky_tip'], 'palm')
                    dist_time = pd.DataFrame(dist, columns=['dist'])
                    dist_time['time'] = blocks[block].program_time.to_list()
                    dist_time_path = os.path.join(
                        repo_path, 'data', 'raw_blocks', f'ul{sub}', task, cond, 'dist_time')
                    if not os.path.exists(dist_time_path):
                        os.makedirs(dist_time_path)
                    dist_time.to_csv(os.path.join(
                        dist_time_path, f'dist_time_{block}_ul{sub}_{cond}_{cam}_{task}_{side}.csv'))

                elif task == 'ft':
                    dist = hp.calc_distances(blocks[block], 'index_tip', 'thumb_tip')
                    dist_time = pd.DataFrame(dist, columns=['dist'])
                    dist_time['time'] = blocks[block].program_time.to_list()
                    dist_time_path = os.path.join(
                        repo_path, 'data', 'raw_blocks', f'ul{sub}', task, cond, 'dist_time')
                    if not os.path.exists(dist_time_path):
                        os.makedirs(dist_time_path)
                    dist_time.to_csv(os.path.join(
                        dist_time_path, f'dist_time_{block}_ul{sub}_{cond}_{cam}_{task}_{side}.csv'))

                elif task == 'ps':
                    dist = hp.PS_ang(blocks[block], 'thumb_tip', 'middle_tip', 'palm')
                    dist_time = pd.DataFrame(dist, columns=['dist'])
                    dist_time['time'] = blocks[block].program_time.to_list()
                    dist_time_path = os.path.join(
                        repo_path, 'data', 'raw_blocks', f'ul{sub}', task, cond, 'dist_time')
                    if not os.path.exists(dist_time_path):
                        os.makedirs(dist_time_path)
                    dist_time.to_csv(os.path.join(
                        dist_time_path, f'dist_time_{block}_ul{sub}_{cond}_{cam}_{task}_{side}.csv'))

                else:
                    # function for postural tremor
                    print('postural tremor was not analysed yet')

TypeError: get_data() missing 1 required positional argument: 'folder'

Load blocks, calculate distances and create dist-time .csv files

In [12]:
for cond in ls_cond:
    files = os.listdir(os.path.join(repo_path, 'data',
                       'raw_blocks', sub, task, cond, 'blocks'))

    for file in files:
        # Load blocks from patients' blocks dir
        block = pd.read_csv(os.path.join(
            repo_path, 'data', 'raw_blocks', sub, task, cond, 'blocks', file), index_col=0)

        if task == 'oc':
            dist = hp.calc_distances(
                block, ['index_tip', 'middle_tip', 'ring_tip', 'pinky_tip'], 'palm')
        elif task == 'ft':
            dist = hp.calc_distances(block, 'index_tip', 'thumb_tip')
        elif task == 'ps':
            dist = hp.PS_ang(block, 'thumb_tip', 'middle_tip', 'palm')
        else:
            # function for postural tremor
            print('postural tremor was not analysed yet')

        dist_time = pd.DataFrame(dist, columns=['dist'])
        dist_time['time'] = block.program_time.to_list()

        dist_time_path = os.path.join(
            repo_path, 'data', 'raw_blocks', sub, task, cond, 'dist_time')
        if not os.path.exists(dist_time_path):
            os.makedirs(dist_time_path)
        dist_time.to_csv(os.path.join(
            dist_time_path, f'dist_time_{file[:2]}_{sub}_{cond}_{cam}_{task}_{side}.csv'))

